In [31]:
import pandas as pd
import gzip
import json
import re
import datetime
import numpy as np
from random import randint
from collections import Counter

In [2]:
class Coinmarketcap(object):
    @staticmethod
    def load_data(source_datapath):
        by_date_data = {}
        all_data = pd.read_csv(source_datapath, sep='\t', header=0, encoding="utf-8", dtype={'Market Cap': np.float64, 'Total Market Cap': np.float64})
        all_data["Market Cap Share"] = all_data["Market Cap"]/all_data["Total Market Cap"]
        for index, value in all_data['Date'].items():
            date = value
            symbol = all_data['Symbol'][index]
            price = all_data['Price'][index]
            marketcap = all_data['Market Cap'][index]
            position = all_data['Pos'][index]
            if by_date_data.get(date) is None:
                by_date_data[date] = {'symbols': {}, 'positions': []}
            by_date_data[date]['symbols'][symbol] = {
                "price": price,
                "marketcap": marketcap
            }
            by_date_data[date]['positions'].append([symbol, position])
        for date in by_date_data.keys():
            by_date_data[date]['positions'] = [el[0] for el in sorted(by_date_data[date]['positions'], key=lambda x: x[1])]
        return all_data, by_date_data

    def __init__(self, source_datapath):
        self.data, self.by_date_data = self.load_data(source_datapath)
    
    def get_coin_price(self, date, coin_symbol):
        try:
            return float(self.by_date_data[date]['symbols'][coin_symbol]['price'])
        except:
            #print("ERROR: price not found for coin %s (%s)" % (coin_symbol, date))
            return 0

    def coins_to_usd(self, date, coins):
        amount_usd = 0
        for coin_symbol, amount in coins.items():
            coin_usd = self.get_coin_price(date, symbol)
            amount_usd += amount * coin_usd
        return amount_usd
    
    def buy_coin(self, date, symbol, amount_usd):
        try:
            return amount_usd/self.get_coin_price(date, symbol)
        except:
            print(date, symbol, amount_usd)
            raise Exception("")
    
    def buy_coins(self, date, symbols, amount_usd, strategy="even"):
        symbols_length = len(symbols)
        coins = {}
        if strategy == "market_cap":
            coins_market_cap = {}
            total_market_cap = 0
            for coin_symbol in symbols:
                coins_market_cap[coin_symbol] = self.get_coin_market_cap(date, coin_symbol)
                total_market_cap += coins_market_cap[coin_symbol]
        for coin_symbol in symbols:
            coin_amount_usd = 0
            if strategy == "even":
                coin_amount_usd = (amount_usd/symbols_length)
            elif strategy == "market_cap":
                coin_amount_usd = (amount_usd*(coins_market_cap[coin_symbol]/total_market_cap))
            coins[coin_symbol] = self.buy_coin(date, coin_symbol, coin_amount_usd)
        return coins
    
    def sell_coins(self, date, coins):
        amount_usd = 0
        for coin_symbol, coin_amount in coins.items():
            coin_price = self.get_coin_price(date, coin_symbol)
            amount_usd += coin_price*coin_amount
        return amount_usd
    
    def get_coin_market_cap(self, date, coin_symbol):
        return float(self.by_date_data[date]['symbols'][coin_symbol]['marketcap'])
            
    def get_positions_range(self, date, first_position, last_position):
        return self.by_date_data[date]['positions'][first_position-1:last_position]
    
    def dates(self):
        return sorted(self.by_date_data.keys())
    
    def print_returns(self, initial_amount, initial_date, final_amount, final_date):
        print("%s: %s USD" % (initial_date, round(initial_amount, 2)))
        print("%s: %s USD" % (end_date, round(final_amount, 2)))
        print("----------")
        print("%sX" % round(final_amount/initial_amount, 1))
        

In [33]:
class InvestStrategies(object):
    def __init__(self, market, options):
        self.market = market
        self.options = options
        self.dates = self.market.dates()

    def buy(self, amount, date):
        raise Exception("Implementation needed")
        

    def get_random_start_end_date(self):
        min_week_duration = self.options.get('min_week_duration') or 1
        min_start_date = self.options.get('min_start_date', self.dates[0])
        max_end_date = self.options.get('max_end_date', self.dates[-1])
        
        min_week_index = min([index for index, date in enumerate(self.dates) if min_start_date <= date])
        max_week_index = max([index for index, date in enumerate(self.dates) if max_end_date >= date])
        
        random_start_index = randint(min_week_index, max_week_index - min_week_duration)
        random_end_index = randint(random_start_index + min_week_duration, max_week_index)
        duration = random_end_index - random_start_index
        return duration, random_start_index, random_end_index

    def buy_top_x(self, date, amount, top_x_first_pos, top_x_last_pos, strategy, exclude=None):
        if exclude is None:
            exclude = []
        symbols_to_buy = self.market.get_positions_range(date, top_x_first_pos, top_x_last_pos)
        symbols_to_buy = [symbol for symbol in symbols_to_buy if symbol not in exclude]
        return self.market.buy_coins(date, list(symbols_to_buy), amount, strategy=strategy)
    
    def add_coins_list(self, coins_group):
        coins_keys = set([
            coin_key
            for coins_data in coins_group
            for coin_key in coins_data.keys()
            
        ])
        coins = {}
        for coins_data in coins_group:
            for coin_key in coins_keys:
                if coins_data.get(coin_key):
                    if coins.get(coin_key) is None:
                        coins[coin_key] = 0
                    coins[coin_key] += coins_data[coin_key]
        return coins

    def run_strategy(self, initial_amount, balance_x_weeks=0):
        iterations = self.options.get('iterations', 10)
        results = {
            "min_date": "",
            "max_date": "",
            "min_duration": "",
            "max_duration": "",
            "values": [],
            "bought_coins": []
        }
        bought_coins = []
        for i in range(0, iterations):
            weeks_duration, start_date_index, end_date_index = self.get_random_start_end_date()
            if not results['max_duration'] or results['max_duration'] < weeks_duration:
                results['max_duration'] = weeks_duration
            if not results['min_duration'] or results['min_duration'] > weeks_duration:
                results['min_duration'] = weeks_duration
            
            if not results['min_date'] or results['min_date'] > self.dates[start_date_index]:
                results['min_date'] = self.dates[start_date_index]
            if not results['max_date'] or results['max_date'] < self.dates[end_date_index]:
                results['max_date'] = self.dates[end_date_index]
            
            amount = initial_amount
            if balance_x_weeks > 0:
                first_date = True
                coins = {}
                for date in self.dates[start_date_index:end_date_index:balance_x_weeks]:
                    if not first_date:
                        amount = market.sell_coins(date, coins)
                    else:
                        first_date = False
                    coins = self.buy(amount, date)
                    bought_coins += coins.keys()
                amount = market.sell_coins(date, coins)
            else:
                coins = self.buy(amount, self.dates[start_date_index])
                bought_coins += coins.keys()
                amount = market.sell_coins(self.dates[end_date_index], coins)
            
            results['values'].append(round(100.0*(amount/initial_amount - 1)/weeks_duration, 3))

        results['iterations'] = iterations
        results['values'] = sorted(results['values'])
        results['mean'] = round(np.mean(results['values']), 2)
        results['median'] = round(np.median(results['values']), 2)
        results['min_rate'] = round(np.min(results['values']), 2)
        results['max_rate'] = round(np.max(results['values']), 2)
        results['bought_coins'] = Counter(bought_coins)
        return results
    
    def print_results(self, results):
        print("\n\n%s:\n" % self.name())
        for key in [
            'min_date', 'max_date', 'min_duration', 'max_duration', 'iterations',
            'min_rate', 'max_rate', 'mean', 'median', 'bought_coins']:
            if key in ['min_rate', 'max_rate', 'mean', 'median']:
                print("%s: %s%%" % (key, results[key]))
            else:
                print("%s: %s" % (key, results[key]))
        
        
        

# Initialize

In [29]:
market = Coinmarketcap("data/coinmarketcap_formatted.tsv")
strategy_options = {
    'min_week_duration': 16,
    'min_start_date': '2015-09-01',
    'iterations': 1000
}

We will implement several strategies, and run them over several dates, from not before 2015-09-01 and not after 2017-04-30. We will run 1000 iterations with a minimun legth of 16 weeks (from first investment and last)

In [34]:
class OnlyBitcoin(InvestStrategies):
    def buy(self, amount, date):
        return self.buy_top_x(date, amount, top_x_first_pos=1, top_x_last_pos=1, strategy="even")

    def name(self):
        return "OnlyBitcoin"

    
class Top5Evenly(InvestStrategies):
    def buy(self, amount, date):
        return self.buy_top_x(date, amount, top_x_first_pos=1, top_x_last_pos=5, strategy="even")

    def name(self):
        return "Top 5 in same proportion"

class Top5MarketCap(InvestStrategies):
    def buy(self, amount, date):
        return self.buy_top_x(date, amount, top_x_first_pos=1, top_x_last_pos=5, strategy="market_cap")

    def name(self):
        return "Top 5 by market cap"

class Top1_50_Top2_5_50(InvestStrategies):
    def buy(self, amount, date):
        top1_coins = self.buy_top_x(date, 1.0*amount/2, top_x_first_pos=1, top_x_last_pos=1, strategy="even")
        top2_5_coins = self.buy_top_x(date, 1.0*amount/2, top_x_first_pos=2, top_x_last_pos=5, strategy="even")
        coins = {**top1_coins, **top2_5_coins}
        return coins

    def name(self):
        return "Top1 50%, Top2-5 50% by marketcap"

class TopX(InvestStrategies):
    def buy(self, amount, date):
        coins_group = []
        coins_group.append(
            self.buy_top_x(date, 0.4*amount, top_x_first_pos=1, top_x_last_pos=1, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.3*amount, top_x_first_pos=2, top_x_last_pos=2, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.2*amount, top_x_first_pos=3, top_x_last_pos=5, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.1*amount, top_x_first_pos=3, top_x_last_pos=10, strategy="even")
        )
        coins = self.add_coins_list(coins_group)
        return coins

    def name(self):
        return "Top1 40%, TOP2 30%, Top3-5 20% evenly, Top3-10 20% evenly"

    
class TopY(InvestStrategies):
    def buy(self, amount, date):
        coins_group = []
        coins_group.append(
            self.buy_top_x(date, 0.35*amount, top_x_first_pos=1, top_x_last_pos=1, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.25*amount, top_x_first_pos=2, top_x_last_pos=2, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.4*amount, top_x_first_pos=3, top_x_last_pos=10, strategy="market_cap")
        )
        coins = self.add_coins_list(coins_group)
        return coins

    def name(self):
        return "Top1 35%, TOP2 25%, Top3-10 40% market_cap"
    
    
class TopZ(InvestStrategies):
    def buy(self, amount, date):
        coins_group = []
        coins_group.append(
            {"BTC": self.market.buy_coin(date, "BTC", amount*0.35)}
        )
        coins_group.append(
            {"ETH": self.market.buy_coin(date, "ETH", amount*0.25)}
        )
        coins_group.append(
            {"DASH": self.market.buy_coin(date, "DASH", amount*0.05)}
        )
        coins_group.append(
            {"XRP": self.market.buy_coin(date, "XRP", amount*0.05)}
        )
        coins_group.append(
            self.buy_top_x(
                date, 0.3*amount, top_x_first_pos=3, top_x_last_pos=10,
                strategy="even", exclude=["BTC", "ETH", "DASH", "XRP"]
            )
        )
        coins = self.add_coins_list(coins_group)
        return coins

    def name(self):
        return "BTC 35%, ETH 25%, DASH 5%, RIPPLE 5%, Top3-10 20% evenly (Without already bought). Could have a cherry picking problem"


    

for class_ in [OnlyBitcoin, Top5Evenly, Top5MarketCap, Top1_50_Top2_5_50, TopX, TopY, TopZ]:
    inv = class_(market, strategy_options)
    results = inv.run_strategy(1000, balance_x_weeks=1)
    inv.print_results(results)




OnlyBitcoin:

min_date: 2015-09-06
max_date: 2017-04-30
min_duration: 16
max_duration: 85
iterations: 1000
min_rate: -0.37%
max_rate: 5.78%
mean: 2.58%
median: 2.64%
bought_coins: Counter({'BTC': 33213})


Top 5 in same proportion:

min_date: 2015-09-06
max_date: 2017-04-30
min_duration: 16
max_duration: 80
iterations: 1000
min_rate: -1.65%
max_rate: 18.76%
mean: 4.23%
median: 2.45%
bought_coins: Counter({'BTC': 32786, 'ETH': 32786, 'XRP': 32786, 'LTC': 31525, 'XMR': 11746, 'DASH': 9758, 'ETC': 4103, 'DAO': 2982, 'STEEM': 2701, 'DOGE': 1473, 'MAID': 1069, 'BTS': 215})


Top 5 by market cap:

min_date: 2015-09-06
max_date: 2017-04-30
min_duration: 16
max_duration: 84
iterations: 1000
min_rate: -0.92%
max_rate: 5.95%
mean: 2.65%
median: 2.68%
bought_coins: Counter({'BTC': 33432, 'ETH': 33432, 'XRP': 33432, 'LTC': 32162, 'XMR': 11714, 'DASH': 10089, 'ETC': 4106, 'DAO': 3112, 'STEEM': 2778, 'DOGE': 1521, 'MAID': 1130, 'BTS': 252})


Top1 50%, Top2-5 50% by marketcap:

min_date: 2015-09-0

In [27]:
""

''